# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></div><div class="lev1 toc-item"><a href="#Counting-$|U|$" data-toc-modified-id="Counting-$|U|$-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Counting <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-2-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mo stretchy=&quot;false&quot;>|</mo></mrow><mi>U</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mo stretchy=&quot;false&quot;>|</mo></mrow></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-6" role="math" style="width: 1.42em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.168em; height: 0px; font-size: 122%;"><span style="position: absolute; clip: rect(1.735em 1001.11em 2.618em -999.998em); top: -2.489em; left: 0.002em;"><span class="mrow" id="MathJax-Span-7"><span class="texatom" id="MathJax-Span-8"><span class="mrow" id="MathJax-Span-9"><span class="mo" id="MathJax-Span-10" style="font-family: STIXMathJax_Main;">|</span></span></span><span class="mi" id="MathJax-Span-11" style="font-family: STIXMathJax_Main-italic;">U<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.033em;"></span></span><span class="texatom" id="MathJax-Span-12"><span class="mrow" id="MathJax-Span-13"><span class="mo" id="MathJax-Span-14" style="font-family: STIXMathJax_Main;">|</span></span></span></span><span style="display: inline-block; width: 0px; height: 2.492em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.075em; border-left: 0px solid; width: 0px; height: 0.925em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">|</mo></mrow><mi>U</mi><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">|</mo></mrow></math></span></span><script type="math/tex" id="MathJax-Element-2">|U|</script></a></div>

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pdmse

trying alternative import for spgl1...success!


In [3]:
from scipy.stats import norm, beta

# Description

In this document, we create several empirical approximations related to quantities computed from order statistics [of normal-random and uniform distributions].
In particular, let $z \in \mathbb{R}^n$ have iid standard normal entries,
$$
z_j \overset{iid}{\sim} \mathcal{N}(0,1), \quad j \in [n]
$$ 
and denote the soft-thresholding operator by 
$$
S_\lambda(y) = \mathrm{sgn}(y) \max(0, |y| - \lambda). 
$$
We are looking to prove that if 
$$
\langle S_\lambda (z), z \rangle \geq \sqrt n
$$
then 
$$
\|S_\lambda (z) \|_2 \geq n^p
$$
for some $p > 0$. We'll be looking for something like $p^{1/3}$ but anything will do. 

Without loss of generality, just assume that $z_j$ are all positive; this changes nothing, 
because we could just analyze a positive and negative $z$ with no difference to the result. 
First note that
$$
\langle S_\lambda(z), z\rangle 
= \langle (z- \lambda)_U, z \rangle 
= \langle z_U, z \rangle - \lambda \langle \mathbb{1}_U, z\rangle
= \|z_U\|_2^2 - \lambda \|z_U\|_1 
$$
and secondly, note 
$$
\|S_\lambda (z) \|_2^2
= \langle (z-\lambda)_U, (z - \lambda)_U\rangle
= \|z_U\|_2^2 - 2 \lambda \|z_U\|_1 + \lambda^2 |U|
$$

# Counting $|U|$

In [60]:
z = np.random.randn(100, 10)
zu = pdmse.softThresh(z, 1)

In [71]:
from pprint import pprint

In [65]:
zxzu = np.inner(z.T, zu.T)

In [102]:
np.einsum('i...,i...', z, zu)

array([ 29.67850221,  19.44799413,  27.26696964,  31.66462721,
        34.7033675 ,  38.49565768,  42.67168926,  27.11633388,
        25.67609335,  22.00576809])

In [122]:
def orderStatNorm_Empirical(n=1000, thresh=1, k=1000, ord=2):
    """
    orderStatNorm_Empirical(n=1000, thresh=1, k=1000, ord=2)
    computes the norm of the values of z which are greater in magnitude than thresh.
    In particular, define for a normal random vector z
      U = { j : |z[j]| > thresh }
      y = (z[j] : j \in U) \in R^|U|
    Then this function approximates
      rho = norm(y, ord)
    by computing the sample norm
      rho_bar = mean((norm(y_ell, ord), ell = 1, ..., k));
    and also returns the proportion
      kappa = #{ ell : <z_ell, zu_ell> >= sqrt(n) }/k
    of the sample that was used in the computation. This is 
    because only the subsample given by the set in the 
    definition of kappa is used in the computation of the mean.

    Input
    =====
         n : the dimension of the random normal vector
    thresh : the threshold value for determining for which sub-vector to compute the norm
         k : the number of random vectors for which to compute the norm for (the mean is returned)
       ord : the norm to compute

    Output
    ======
    rho_bar : the mean norm of the vector z_U (norm_zu.mean())
      kappa : the proportion of the sample of random vectors that 
              was valid for use in the computation of rho_bar
    """
    z = np.random.randn(n, k)
    zu = pdmse.softThresh(z, thresh)  # zu = sgn(z) * max(0, |z| - thresh)
    inner_products = np.einsum('i...,i...', z, zu)  # <z_j, zu_j> for j = 1, ..., k
    condition = inner_products > np.sqrt(n)
    norm_zu = np.linalg.norm(z[:, condition], ord=ord, axis=0)
    return (norm_zu.mean(), condition.sum()/k)

In [139]:
orderStatNorm_Empirical(thresh=2, ord=1)

(798.56079158809712, 0.92900000000000005)

In [59]:
def aboveThresh_Empirical(n=1000, thresh=1, k=1000):
    z = np.random.randn(n*k)
    zu = pdmse.softThresh(z, thresh)
    return (np.abs(zu)>0).sum()/k


def aboveThresh_Analytic(n=1000, thresh=1):
    return np.floor((n+1) * 2 * norm.cdf(-thresh))

In [53]:
aboveThresh_Analytic()

316.0

In [58]:
aboveThresh_Empirical()

316.80900000000003

In [18]:
np.ceil(norm.cdf(2.5)*(10000+1))

9939.0

In [21]:
n=10000

In [30]:
(n+1) - np.ceil((n+1) * norm.cdf(2.5))

62.0

In [31]:
np.floor((1-norm.cdf(2.5))*(n+1))

62.0